<a href="https://colab.research.google.com/github/amirpaia/blenderbot/blob/main/evaluation_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu May 19 13:38:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
# !pip uninstall -q parlai
!pip install -q parlai
!pip install transformers

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
     |████████████████████████████████| 95 kB 6.0 MB/s 
     |████████████████████████████████| 175 kB 87.3 MB/s 
     |████████████████████████████████| 132 kB 76.3 MB/s 
     |████████████████████████████████| 248 kB 96.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 48 kB 5.8 MB/s 
     |████████████████████████████████| 40 kB 6.8 MB/s 
     |████████████████████████████████| 138 kB 90.4 MB/s 
     |████████████████████████████████| 181 kB 91.9 MB/s 
     |████████████████████████████████| 74 kB 4.2 MB/s 
     |████████████████████████████████| 56 kB 5.8 MB/s 
     |████████████████████████████████| 147 kB 102.2 MB/s 
     |████████████████████████████████| 6.6 MB 75.7 MB/s 
     |████████████████████████████████| 208 kB 76.4 MB/s 
     |███████████████████████

In [3]:
def colored(r, g, b, text):
    return "\033[38;2;{};{};{}m{} \033[38;2;255;255;255m".format(r, g, b, text)


# Grid Search

In [11]:
model_path = "/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m/"
from sklearn.model_selection import ParameterGrid

grid = [
    {
        'task': ['french_blended_skill_talk'], 
        'model_file': [f'{model_path}model'],
        'datatype': ['valid'],
        'num_examples' : [20],
        'skip_generation': [False],
        'report_filename' : ['report.json'],
        'world_logs' : ['world-log.json'],
        'display_examples' : [True],

        'beam_context_block_ngram': [2,3],
        'beam_block_ngram': [2,3],
        'beam_min_length': [10, 20, 30],
        'beam_size': [10, 20],
        'beam_length_penalty': [0.7, 0.8, 1],
        'topp': [0.3, 0.9],
     
        'inference': ['beam', 'greedy'],
        'temperature': [0.5, 0.8],
    },
    # {
    #     'task': ['french_blended_skill_talk'], 
    #     'model_file': [f'{model_path}model'],
    #     'datatype': ['valid'],
    #     'num_examples' : [2],
    #     'skip_generation': [False],
    #     'world_logs' : ['world-log.json'],
    #     'display_examples' : [True],

    #     'beam_context_block_ngram': [2,3],
    #     'beam_block_ngram': [2,3],
    #     'beam_min_length': [10, 20, 30],
    #     'beam_size': [10, 20],
    #     'beam_length_penalty': [0.7, 0.8, 1],
    #     'topp': [0.3, 0.9],
     
    #     'inference': ['topk'],   
    #     'temperature': [0.5, 0.8],
    #     'topk': [10,20, 30]
    # }
]
combinations = list( ParameterGrid(grid))
print(len(combinations))


from parlai.scripts.display_model import DisplayModel
from parlai.scripts.eval_model import EvalModel
import json
import time

# output_path = "/content/drive/MyDrive/colabs/blender-models/output-of-gridsearch-topk-on-multitask-400m/"
output_path = "/content/drive/MyDrive/colabs/blender-models/output-of-gridsearch-beam+greedy-on-multitask-400m/"
for index, param in enumerate(combinations):
    start_time = time.time()

    if index > 0 :
        param['world_logs']= f'world-log-{index}'
        result = EvalModel.main(**param)
        
        with open(f'{output_path}world-log-{index}.jsonl', 'a') as f:
            f.write( "\n" + json.dumps(param) )

        print(index, round(time.time() - start_time, 2), "\n")

576
13:57:56 | Overriding opt["datatype"] to valid (previously: train)
13:57:56 | Overriding opt["beam_block_ngram"] to 2 (previously: -1)
13:57:56 | Overriding opt["beam_context_block_ngram"] to 2 (previously: -1)
13:57:56 | Overriding opt["beam_length_penalty"] to 0.7 (previously: 0.65)
13:57:56 | Overriding opt["beam_min_length"] to 10 (previously: 1)
13:57:56 | Overriding opt["beam_size"] to 10 (previously: 1)
13:57:56 | Overriding opt["display_examples"] to True (previously: False)
13:57:56 | Overriding opt["inference"] to beam (previously: greedy)
13:57:56 | Overriding opt["skip_generation"] to False (previously: True)
13:57:56 | Overriding opt["task"] to french_blended_skill_talk (previously: french_blended_skill_talk,french_xpersona,french_empathetic_dialogues)
13:57:56 | Overriding opt["temperature"] to 0.5 (previously: 1.0)
13:57:56 | Overriding opt["topp"] to 0.3 (previously: 0.9)
13:57:56 | Overriding opt["world_logs"] to world-log-0 (previously: )
13:57:56 | Using CUDA
13:

/usr/local/lib/python3.7/dist-packages/parlai/core/torch_generator_agent.py:1728: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  hyp_ids = best_idxs // voc_size


[french_blended_skill_talk]: Cela semble dangereux. Cela vaut-il la peine de faire un travail aussi dangereux ?
[eval_labels]: Wekk, c'est bon si vous êtes bien formé.  Il y a trois niveaux 
   [TransformerGenerator]: Je suis désolé d'entendre ça.
~~
[french_blended_skill_talk]: A quel niveau êtes-vous ?
[eval_labels]: J'ai reçu une formation sur le tas lorsque j'ai commencé à travailler.
   [TransformerGenerator]: Je suis désolé d'entendre ça.
~~
[french_blended_skill_talk]: C'est génial ! Depuis combien de temps faites-vous ce travail ? 
[eval_labels]: Depuis un bon nombre d'années maintenant.
   [TransformerGenerator]: Je suis désolé d'entendre ça.
~~
[french_blended_skill_talk]: On dirait que ça peut être un travail dangereux parfois.
[eval_labels]: C'est vrai, surtout si vous ne prenez pas les mesures appropriées.
   [TransformerGenerator]: Ça a l'air délicieux !
~~
[french_blended_skill_talk]: Eh bien, vous êtes entré dans un métier qui vous garantira toujours du travail.
[eval_l

/usr/local/lib/python3.7/dist-packages/parlai/core/torch_generator_agent.py:1728: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  hyp_ids = best_idxs // voc_size


[french_blended_skill_talk]: C'est vrai, surtout si vous ne prenez pas les mesures appropriées.
[eval_labels]: Eh bien, vous êtes entré dans un métier qui vous garantira toujours du travail.
   [TransformerGenerator]: Ça a l'air amusant ! 
~~
[french_blended_skill_talk]: C'est vrai, surtout dans cette économie
[eval_labels]: J'ai moi-même un emploi assez stable, mais surtout du travail de bureau.
   [TransformerGenerator]: Ça a l'air délicieux.
~~
[french_blended_skill_talk]: Est-ce que ça vous plaît ?
[eval_labels]: Je le fais vraiment, mais parfois j'aimerais être plus actif et debout.
   [TransformerGenerator]: Non, je suis sûr que je vais aujourd'hui.
- - - - - - - END OF EPISODE - - - - - - - - - -
~~
[french_blended_skill_talk]: Je me sens très bien dans ma relation avec mon partenaire. Mon partenaire apprécie mon opinion sur les choses.
[eval_labels]: ok, si doux
   [TransformerGenerator]: Je suis désolé d'entendre ça. C'est génial !
~~
[french_blended_skill_talk]: Et vous, quel

NameError: ignored

In [ ]:
import json
dict_ppl_of_files = {}
for i in range(0, 863,1):
    output_file = f"/content/drive/MyDrive/colabs/blender-models/output-of-gridsearch-topk-on-multitask-400m/world-log-{i}.jsonl"
    with open(output_file) as f:
        lines = f.readlines();
    list_ppl_of_dialogs = []
    for line in lines:
        if (line != "\n"):
            obj = json.loads(line)
            if "dialog" in obj.keys():
                # print("\n\nNew Dialog. Number of turns:", len(obj['dialog']), "|", len(obj['dialog'][0]), "\n")
                ppl_of_dialog = sum([d[1]['metrics']['ppl'] for d in obj['dialog']]) / len(obj['dialog'])
                list_ppl_of_dialogs.append(ppl_of_dialog)

                print(list_ppl_of_dialogs)
    ppl_of_file = sum(list_ppl_of_dialogs) / len(list_ppl_of_dialogs)
    print(output_file, ppl_of_file)

    dict_ppl_of_files[str(i)] = ppl_of_file
    ppl_of_file = 0
                # for d in obj['dialog']:
                #     text = d[0]['text']
                #     label = d[0]['eval_labels'][0]
                #     prediction = d[1]['text']
                #     beam_score = d[1]['beam_texts'][0][1]
                #     ppl = d[1]['metrics']['ppl']
                #     # print(text, label, prediction, beam_score)
                #     print("[text]:", text)
                #     print("    [label]:", colored(0,200,0, label))
                #     print("\t[prediction]:", colored(230,0,230, prediction), beam_score, ppl)
            # elif "task" in obj.keys():
            #     print("param")
    if i>2 : break
# print(dict_ppl_of_files)

# {k: v for k, v in sorted(dict_ppl_of_files.items(), key=lambda item: item[1])}


# Perplexity

In [ ]:
!pip install transformers
!pip install torch
!pip install sacremoses

In [ ]:
!pip install sentencepiece 

     |████████████████████████████████| 1.2 MB 7.0 MB/s 


In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
import numpy as np

# ['flaubert/flaubert_small_cased', 'flaubert/flaubert_base_uncased', 'flaubert/flaubert_base_cased', 'flaubert/flaubert_large_cased']
# model_name = 'flaubert/flaubert_small_cased'
# model_name = 'flaubert/flaubert_base_uncased'
# model_name = 'flaubert/flaubert_base_cased'
model_name = 'flaubert/flaubert_large_cased'

# model_name = 'camembert-base'


model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def score(model, tokenizer, sentence):
    tensor_input = tokenizer.encode(sentence, return_tensors='pt')
    repeat_input = tensor_input.repeat(tensor_input.size(-1) - 2, 1)
    mask = torch.ones(tensor_input.size(-1) - 1).diag(1)[:-2]
    masked_input = repeat_input.masked_fill(mask == 1, tokenizer.mask_token_id)
    labels = repeat_input.masked_fill( masked_input != tokenizer.mask_token_id, -100)
    with torch.inference_mode():
        loss = model(masked_input, labels=labels).loss
    return np.exp(loss.item())

print(score(sentence='le chat mange une pomme.', model=model, tokenizer=tokenizer)) 
print(score(sentence='le garçon mange une pomme.', model=model, tokenizer=tokenizer)) 

Downloading:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.49M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/875k [00:00<?, ?B/s]

25.598162479248977
31.979964160350367


In [4]:
import json
dict_ppl_of_files = {}
output_path = "/content/drive/MyDrive/colabs/blender-models/output-files/"
with open(output_path + "output-perplexity.txt") as f:
    text = f.read()
    dict_ppl_of_files = json.loads(text)
print(len(dict_ppl_of_files))

863


In [7]:
import pandas as pd
import numpy as np
df = pd.DataFrame.from_dict(dict_ppl_of_files, orient='index', columns=['d1_with', 'd1_wo', 'd2_with', 'd2_wo', "d3_with", "d3_wo"])
# display(df.head())

df['avg_with_outliers'] = df.apply(lambda x: np.mean( [float(x['d1_with']), float(x['d2_with']), float(x['d3_with'])] ), axis=1)
df['avg_wo_outliers'] = df.apply(lambda x: np.mean( [float(x['d1_wo']), float(x['d2_wo']), float(x['d3_wo'])] ), axis=1)

df_top20 = df.sort_values(by=['avg_wo_outliers']).head(20)
display(df_top20)

top20_filenames = df_top20.index.values.tolist()
print(top20_filenames)

,d1_with,d1_wo,d2_with,d2_wo,d3_with,d3_wo,avg_with_outliers,avg_wo_outliers
world-log-375.jsonl,18.885397,14.701161,41.521657,14.771811,16.662099,10.636507,25.689718,13.369826
world-log-373.jsonl,38.856314,16.120551,66.41695,22.061378,38.054917,11.029829,47.776060,16.403919
world-log-256.jsonl,89.36714328956006,17.814023227945324,50.17399382951289,13.45247999412779,32.95077902318858,18.726104204493137,57.497305,16.664202
world-log-829.jsonl,24.133294,24.133294,15.635502,10.252311,22.576355,16.112056,20.781717,16.832554
world-log-831.jsonl,26.553708,18.582489,46.618964,17.189357,19.029973,19.029973,30.734215,18.267273
world-log-362.jsonl,178.12883,18.224497,50.930059,25.562413,23.206498,11.713098,84.088463,18.500003
world-log-758.jsonl,70.652386,22.339108,111.298848,13.302215,24.818833,19.98571,68.923356,18.542344
world-log-805.jsonl,35.471286,35.471286,38.877906,11.599309,25.143744,10.342167,33.164312,19.137587
world-log-747.jsonl,30.497438,22.702527,30.187938,15.790077,45.82559,19.561715,35.503655,19.351439
world-log-833.jsonl,42.141053,25.886208,29.49642,13.896885,24.990982,18.602639,32.209485,19.461911


['world-log-375.jsonl', 'world-log-373.jsonl', 'world-log-256.jsonl', 'world-log-829.jsonl', 'world-log-831.jsonl', 'world-log-362.jsonl', 'world-log-758.jsonl', 'world-log-805.jsonl', 'world-log-747.jsonl', 'world-log-833.jsonl', 'world-log-220.jsonl', 'world-log-305.jsonl', 'world-log-385.jsonl', 'world-log-685.jsonl', 'world-log-854.jsonl', 'world-log-855.jsonl', 'world-log-708.jsonl', 'world-log-374.jsonl', 'world-log-324.jsonl', 'world-log-672.jsonl']


There are 3 dialog in each file, this piece of code calculates perplexity for each dialog and saves it in a text file

In [ ]:
def reject_outliers(data, m=2):
    mean = np.mean(data)
    std = np.std(data)
    return [a for a in data if (abs(a - mean) < m * std)]

In [ ]:
import json
import time
output_path = "/content/drive/MyDrive/colabs/blender-models/output-files/"

for i in range(len(dict_ppl_of_files), 863,1):
# for i in range(322, 323,1):
    output_file = f"world-log-{i}.jsonl"
    with open(output_path + output_file) as f:
        lines = f.readlines()

    for i, line in enumerate(lines):
        list_ppl_of_dialogs = []
        if (line != "\n"):
            obj = json.loads(line)
            if "dialog" in obj.keys():
                start_time = time.time()
                for d in obj['dialog']:
                    text = d[0]['text']
                    label = d[0]['eval_labels'][0]
                    prediction = d[1]['text']
                    # print(colored(200,200,200,  f"[text]: {text}\t{score(sentence= text, model= model, tokenizer= tokenizer)}" ))
                    # print(colored(255,255,255,  f"    [label]: {label}\t{score(sentence= label, model= model, tokenizer= tokenizer)}"))
                    # print(colored(0,200,0,      f"\t[model]:{prediction}\t{score(sentence= prediction, model= model, tokenizer= tokenizer)}"))
                    prediction_score = score(sentence= prediction, model= model, tokenizer= tokenizer)
                    list_ppl_of_dialogs.append(prediction_score)
                list_ppl_of_dialogs_without_outliers = reject_outliers(list_ppl_of_dialogs)
                # print("list_ppl_of_dialogs: ", list_ppl_of_dialogs)
                # print("list_ppl_of_dialogs_without_outliers: ", list_ppl_of_dialogs_without_outliers)
                if not output_file in dict_ppl_of_files:
                    dict_ppl_of_files[output_file] = [np.mean(list_ppl_of_dialogs)]
                    dict_ppl_of_files[output_file].append(np.mean(list_ppl_of_dialogs_without_outliers))
                else:
                    dict_ppl_of_files[output_file].append(np.mean(list_ppl_of_dialogs))
                    dict_ppl_of_files[output_file].append(np.mean(list_ppl_of_dialogs_without_outliers))

                # dict_ppl_of_files[output_file] = [np.mean(list_ppl_of_dialogs), np.mean(list_ppl_of_dialogs_without_outliers)]
                print(f"{output_file}-{i}", dict_ppl_of_files[output_file], i==2 and time.time() - start_time)




In [ ]:
import json
print(len(dict_ppl_of_files))
for key, value in dict_ppl_of_files.items():
    if (len(value) != 6):
        print(key, value)
# dict_ppl_of_files.pop('world-log-640.jsonl', None)

# print(len(dict_ppl_of_files))

with open(output_path + "output.txt", "w") as f:
    f.write(json.dumps(dict_ppl_of_files))

863


In [ ]:
# print( abs(list_ppl_of_dialogs - np.mean(list_ppl_of_dialogs)) )
# print(np.std(list_ppl_of_dialogs))

print(list_ppl_of_dialogs)
print(reject_outliers(list_ppl_of_dialogs))

[34.10929283343882, 3.988064979280301, 8.034693433758147, 2.7560870710324603, 78.53545557203665, 6.75183203681701, 55.791589633481344]
[34.10929283343882, 3.988064979280301, 8.034693433758147, 2.7560870710324603, 78.53545557203665, 6.75183203681701, 55.791589633481344]


### Claculating perplexity on one XPersona dialog

In [ ]:
data_path = "/content/dataset_french_xpersona/"
!rm -R $data_path
!mkdir $data_path
!git clone https://github.com/HLTCHKUST/Xpersona.git

import json
import pandas as pd

with open('Xpersona/dataset/Fr_persona_train_corrected.json','r') as f:
   train_data = json.load(f)

dialogs_train = pd.DataFrame(train_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_valid_human_annotated.json','r') as f:
   valid_data = json.load(f)

dialogs_valid = pd.DataFrame(valid_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_test_human_annotated.json','r') as f:
   text_data = json.load(f)

dialogs_test = pd.DataFrame(text_data)['dialogue'].tolist()


import pandas as pd
text = ""
def transfer_dialog(d):
  t = ""
  for i, each_two_turn in enumerate(d):
    u1 = each_two_turn[0]
    u2 = each_two_turn[1]

    if i != (len(d) - 1):
      t += "text:"+u1+"\t"+"labels:"+u2+"\n"
    else:
      t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"

  return t

data_train = ""
for d in dialogs_train:
  data_train += transfer_dialog(d)

# save to file
file_train = open(f"{data_path}train.txt","w")
print(file_train.write(data_train))

data_valid = ""
for d in dialogs_valid:
  data_valid += transfer_dialog(d)

# save to file
file_valid = open(f"{data_path}valid.txt","w")
print(file_valid.write(data_valid))

data_test = ""
for d in dialogs_test:
  data_test += transfer_dialog(d)

# save to file
file_test = open(f"{data_path}test.txt","w")
print(file_test.write(data_test))

rm: cannot remove '/content/dataset_french_xpersona/': No such file or directory
Cloning into 'Xpersona'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 282 (delta 5), reused 4 (delta 4), pack-reused 275
Receiving objects: 100% (282/282), 45.00 MiB | 16.50 MiB/s, done.
Resolving deltas: 100% (95/95), done.
15776427
257114
262425


In [ ]:
data_path = "/content/dataset_french_xpersona/"
output_file = f"{data_path}test.txt"
with open(output_file) as f:
        lines = f.readlines();
# for line in lines[:8]:
for i,line in enumerate(lines[:20]):
    split_line = line.split('\t')
    text, label, episode_done = split_line[0].replace("text:", "").replace("\n",""), \
                                split_line[1].replace("labels:","").replace("\n",""), \
                                True if len(split_line) == 3 and split_line[2] else False
    
    # text = d[0]['text']
    # label = d[0]['eval_labels'][0]
    # prediction = d[1]['text']
    print(colored(200,200,200,  f"[text]: {text}\t{score(sentence= text, model= model, tokenizer= tokenizer)}" ))
    print(colored(255,255,255,  f"    [label]: {label}\t{score(sentence= label, model= model, tokenizer= tokenizer)}"))
    # print(colored(0,200,0,      f"\t[model]:{prediction}\t{score(sentence= prediction, model= model, tokenizer= tokenizer)}"))
    
    if episode_done: break

[text]: bonjour que fais tu aujourd'hui?	23.50969213523441 
    [label]: je vais bien, je viens de quitter le travail et je suis fatigué, j'ai deux emplois.	69.5941526124176 
[text]: je viens de finir de regarder un film d'horreur	12.41011383669815 
    [label]: j'ai plutôt lu, j'ai lu environ 20 livres cette année.	84.6883624717502 
[text]: Hou la la ! j'aime un bon film d'horreur. aimer ce temps plus frais	215.52674788825635 
    [label]: mais un bon film est toujours bon.	24.56072223917881 
[text]: Oui ! mon fils est au premier cycle du secondaire et je viens de le laisser le regarder aussi	9.66275301463691 
    [label]: je travaille aussi dans les films.	51.23069691556294 
[text]: bon ! ! j'ai travaillé dans le domaine des services à la personne	20.93245391765761 
    [label]: oui c'est soigné, je stunt double, c'est tellement amusant et dur de travailler.	1661.2983740969476 
[text]: oui, je parie que vous pouvez vous blesser. ma femme travaille et je reste à la maison	24.705473362

# Diversity

In [ ]:
!pip install nltk

In [ ]:
# from nltk.translate import bleu_score
# from nltk.translate.bleu_score import SmoothingFunction
# import numpy as np

In [ ]:
cand = ["The cat is tired", "The cat is on the mat", "The pigeon sat on the branch"]
values = distinct(cand)
print(f" intra_distinc1: {values[0]}\n", f"intra_distinc2: {values[1]}\n", f"inter_distinc1: {values[2]}\n", f"inter_distinc2: {values[3]}\n")

 intra_distinc1: 0.9999980555596854
 intra_distinc2: 0.9999975555621704
 inter_distinc1: 0.6249996093753066
 inter_distinc2: 0.769230177515325



In [ ]:
def get_list_of_sentences(file_name, label = "    labels:"):
    with open(file_name) as file:
        lines = file.readlines()
    i =0
    dialog_score_pairs = []
    turns = []
    list_diversity_scores = []
    for line in lines:
        if ("NEW EPISODE" in line):
            if len(turns) > 0 : 
                diversity_scores = distinct(turns)
                list_diversity_scores.append(diversity_scores)
                dialog_score_pairs.append([turns, diversity_scores])

            turns = []
        if (line.startswith(label)):
            turns.append(line[len(label)+1:len(line)-3])
    
    average_diversity_score = [
        np.average([item[0] for item in list_diversity_scores]),
        np.average([item[1] for item in list_diversity_scores]),
        np.average([item[2] for item in list_diversity_scores]),
        np.average([item[3] for item in list_diversity_scores])
    ]

    average_of_turns = np.average([len(item[0]) for item in dialog_score_pairs])
    
    return [dialog_score_pairs, average_of_turns, average_diversity_score]

# file_name4 = "models-examples\\04-finetuned-dict-testset-output.txt"
# labels = get_list_of_sentences(file_name4, "    labels:")
# values4 = distinct(labels)
# labels

In [ ]:
file_name4 = "models-examples\\04-finetuned-dict-testset-output.txt"

labels_n_scores = get_list_of_sentences(file_name4)
print("labels:")
print(f"turns average count: {round(labels_n_scores[1],2)}\n",
    f"intra_distinc1: {labels_n_scores[2][0]}\n", 
    f"intra_distinc2: {labels_n_scores[2][1]}\n", 
    f"inter_distinc1: {labels_n_scores[2][2]}\n", 
    f"inter_distinc2: {labels_n_scores[2][3]}\n")


generated_sentences4_n_scores = get_list_of_sentences(file_name4, "     model:")
# print(f"number of generated sentences in 4th model: {len(generated_sentences4)}")

print("4:")
print(f"turns average count: {round(generated_sentences4_n_scores[1],2)}\n",
    f"intra_distinc1: {generated_sentences4_n_scores[2][0]}\n", 
    f"intra_distinc2: {generated_sentences4_n_scores[2][1]}\n", 
    f"inter_distinc1: {generated_sentences4_n_scores[2][2]}\n", 
    f"inter_distinc2: {generated_sentences4_n_scores[2][3]}\n")

file_name5 = "models-examples\\05-finetuned-beam_prmts_infrnc-testset-output.txt"
generated_sentences5_n_scores = get_list_of_sentences(file_name5, "     model:")
# print(f"number of generated sentences in 9th model: {len(generated_sentences9)}")

print("5:")
print(f"turns average count: {round(generated_sentences5_n_scores[1],2)}\n",
    f"intra_distinc1: {generated_sentences5_n_scores[2][0]}\n", 
    f"intra_distinc2: {generated_sentences5_n_scores[2][1]}\n", 
    f"inter_distinc1: {generated_sentences5_n_scores[2][2]}\n", 
    f"inter_distinc2: {generated_sentences5_n_scores[2][3]}\n")

file_name9 = "models-examples\\09-finetuned-400m-topk-5epochs-testset-output.txt"
generated_sentences9_n_scores = get_list_of_sentences(file_name9, "     model:")
# print(f"number of generated sentences in 9th model: {len(generated_sentences9)}")

print("9:")
print(f"turns average count: {round(generated_sentences9_n_scores[1],2)}\n",
    f"intra_distinc1: {generated_sentences9_n_scores[2][0]}\n", 
    f"intra_distinc2: {generated_sentences9_n_scores[2][1]}\n", 
    f"inter_distinc1: {generated_sentences9_n_scores[2][2]}\n", 
    f"inter_distinc2: {generated_sentences9_n_scores[2][3]}\n")

In [15]:
from collections import Counter
def distinct(seqs):
    """ Calculate intra/inter distinct 1/2. """
    batch_size = len(seqs)
    intra_dist1, intra_dist2 = [], []
    unigrams_all, bigrams_all = Counter(), Counter()
    for seq in seqs:
        words = seq.split()
        unigrams = Counter(words)
        bigrams = Counter(zip(words, words[1:]))

        intra_dist1.append((len(unigrams)+1e-12) / (len(words)+1e-5))
        intra_dist2.append((len(bigrams)+1e-12) / (max(0, len(words)-1)+1e-5))

        unigrams_all.update(unigrams)
        bigrams_all.update(bigrams)

    inter_dist1 = (len(unigrams_all)+1e-12) / (sum(unigrams_all.values())+1e-5)
    inter_dist2 = (len(bigrams_all)+1e-12) / (sum(bigrams_all.values())+1e-5)

    intra_dist1 = np.average(intra_dist1)
    intra_dist2 = np.average(intra_dist2)
    return intra_dist1, intra_dist2, inter_dist1, inter_dist2

# cand = ["The cat is tired", "The cat is on the mat", "The pigeon sat on the branch"]
# values = distinct(cand)
# print(f" intra_distinc1: {values[0]}\n", f"intra_distinc2: {values[1]}\n", f"inter_distinc1: {values[2]}\n", f"inter_distinc2: {values[3]}\n")

In [16]:
import json
import time
output_path = "/content/drive/MyDrive/colabs/blender-models/output-files/"

dict_diversity_scores = {}
for filename in top20_filenames:
    with open(output_path + filename) as f:
        lines = f.readlines()
    diversity_scores_of_dialogs = []
    for i, line in enumerate(lines):
        if (line != "\n"):
            obj = json.loads(line)
            if "dialog" in obj.keys():
                start_time = time.time()
                predictions = []
                for d in obj['dialog']:
                    text, label, prediction  = d[0]['text'], d[0]['eval_labels'][0], d[1]['text']
                    predictions.append(prediction)

                diversity_scores = distinct(predictions)
                diversity_scores_of_dialogs.append(diversity_scores)
    dict_diversity_scores[filename] = [
        np.mean([x[0] for x in diversity_scores_of_dialogs]),
        np.mean([x[1] for x in diversity_scores_of_dialogs]),
        np.mean([x[2] for x in diversity_scores_of_dialogs]),
        np.mean([x[3] for x in diversity_scores_of_dialogs]),
    ]

dict_diversity_scores

df_diversity = pd.DataFrame.from_dict(dict_diversity_scores, orient='index', columns=['intra_distinc1', 'intra_distinc2', 'inter_distinc1', 'inter_distinc2'])
df_diversity

,intra_distinc1,intra_distinc2,inter_distinc1,inter_distinc2
world-log-375.jsonl,0.999998,0.999997,0.735397,0.857488
world-log-373.jsonl,0.989857,0.999998,0.644893,0.792121
world-log-256.jsonl,0.966853,0.999999,0.626691,0.786262
world-log-829.jsonl,0.953837,0.991396,0.612779,0.805997
world-log-831.jsonl,0.963296,0.997023,0.652426,0.868059
world-log-362.jsonl,0.988969,0.999998,0.669387,0.841471
world-log-758.jsonl,0.975500,0.999999,0.624556,0.819472
world-log-805.jsonl,0.986280,0.999998,0.611031,0.767642
world-log-747.jsonl,0.966137,0.999999,0.682820,0.878677
world-log-833.jsonl,0.959986,0.999999,0.574417,0.776423


# Coherency ✅

In [18]:
!git lfs install
!git clone https://huggingface.co/Sahajtomar/french_semantic

# !pip uninstall sentence-transformers
!pip install sentence-transformers

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
Cloning into 'french_semantic'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 30 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (30/30), done.
tcmalloc: large alloc 1471086592 bytes == 0x55a4cda5e000 @  0x7f9cb9d282a4 0x55a491637e8f 0x55a491614fcb 0x55a4915c9f33 0x55a49156e22a 0x55a49156e6e6 0x55a49158b451 0x55a49158b9e9 0x55a49158bf13 0x55a491630b82 0x55a4914d2162 0x55a4914b8a65 0x55a4914b9725 0x55a4914b872a 0x7f9cb906ec87 0x55a4914b877a
     |████████████████████████████████| 79 kB 4.7 MB/s 
     |████████████████████████████████| 1.2 MB 40.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=636b68879bec2000aa7f2894b5d09dac1bd964a35f86

In [ ]:
output_lines = []
with open("/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/test.txt") as f:
    output_lines = f.readlines()
print(len(output_lines))

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('Sahajtomar/french_semantic')

In [ ]:
# calculating the baseline
text, label, generated, context = "", "", "", ""
turns = []

list_of_similarity_scores_for_turns_in_a_dialog = []
list_of_similarity_scores_for_every_dialogs = []

for i,line in enumerate(output_lines):
    split_line = line.split('\t')
    text, label, episode_done = split_line[0].replace("text:", "").replace("\n",""), \
                                split_line[1].replace("labels:","").replace("\n",""), \
                                True if len(split_line) == 3 and split_line[2] else False
    # print("%r"%text, "%r"%label, episode_done)
    label = ' '.join(label.split(' ')[-350:])
    embeddings_label = model.encode(label, convert_to_tensor=True)

    turns.append(text)
    context = ' '.join(turns)
    context = ' '.join(context.split(' ')[-350:]) # if the context is so long, truncate it and pick up the latest 350 token
    embeddings_context = model.encode(context, convert_to_tensor=True)

    cosine_similarity = util.pytorch_cos_sim(embeddings_label, embeddings_context)
    list_of_similarity_scores_for_turns_in_a_dialog.append(cosine_similarity.item())

    turns.append(label)
        
    if (episode_done):
        average_similarity_score = sum(list_of_similarity_scores_for_turns_in_a_dialog) / len(list_of_similarity_scores_for_turns_in_a_dialog)
        list_of_similarity_scores_for_every_dialogs.append(average_similarity_score)
        context = ""
        label = ""
        turns = []

        if len(list_of_similarity_scores_for_every_dialogs) % 50 == 0:
            print(f"{i}: Similarity Score for dialog No. {len(list_of_similarity_scores_for_every_dialogs)}:", average_similarity_score)
        # break


print("Count: ", len(list_of_similarity_scores_for_every_dialogs))
similarity_score_baseline = sum(list_of_similarity_scores_for_every_dialogs)/ len(list_of_similarity_scores_for_every_dialogs)
print("Average: ", similarity_score_baseline)
print(list_of_similarity_scores_for_every_dialogs)

In [19]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('Sahajtomar/french_semantic')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/683 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/809k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/933 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/769 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [20]:
import json
import time
output_path = "/content/drive/MyDrive/colabs/blender-models/output-files/"

dict_similarity_scores = {}
for filename in top20_filenames:
    with open(output_path + filename) as f:
        lines = f.readlines()
    similarity_scores_of_dialogs = []
    for i, line in enumerate(lines):
        if (line != "\n"):
            obj = json.loads(line)
            if "dialog" in obj.keys():
                start_time = time.time()
                turns_as_context = []
                list_of_similarity_scores_for_turns_in_a_dialog = []
                for d in obj['dialog']:
                    text, label, prediction  = d[0]['text'], d[0]['eval_labels'][0], d[1]['text']

                    prediction = ' '.join(prediction.split(' ')[-350:])
                    embeddings_prediction = model.encode(prediction, convert_to_tensor=True)

                    turns_as_context.append(text)
                    context = ' '.join(turns_as_context)
                    context = ' '.join(context.split(' ')[-350:]) # if the context is so long, truncate it and pick up the latest 350 token
                    embeddings_context = model.encode(context, convert_to_tensor=True)

                    cosine_similarity = util.pytorch_cos_sim(embeddings_prediction, embeddings_context)
                    list_of_similarity_scores_for_turns_in_a_dialog.append(cosine_similarity.item())

                    turns_as_context.append(label)

                similarity_scores_of_dialogs.append(np.mean(list_of_similarity_scores_for_turns_in_a_dialog))
                # diversity_scores = distinct(predictions)
                # diversity_scores_of_dialogs.append(diversity_scores)
    dict_similarity_scores[filename] = [np.mean(similarity_scores_of_dialogs)]

    
dict_similarity_scores

df_similarity = pd.DataFrame.from_dict(dict_similarity_scores, orient='index', columns=['similarity_score'])
df_similarity

,similarity_score
world-log-375.jsonl,0.175455
world-log-373.jsonl,0.213010
world-log-256.jsonl,0.201530
world-log-829.jsonl,0.209115
world-log-831.jsonl,0.193791
world-log-362.jsonl,0.220024
world-log-758.jsonl,0.246287
world-log-805.jsonl,0.175879
world-log-747.jsonl,0.229194
world-log-833.jsonl,0.213349


In [23]:
df_result = pd.concat([df_top20, df_diversity, df_similarity], axis=1, join='inner')
# display(df_result)
df_result[['avg_wo_outliers', 'intra_distinc2', 'inter_distinc2', 'similarity_score']].sort_values(by=['similarity_score']).head(20)
# df.sort_values(by=['avg_wo_outliers']).head(20)

,avg_wo_outliers,intra_distinc2,inter_distinc2,similarity_score
world-log-375.jsonl,13.369826,0.999997,0.857488,0.175455
world-log-805.jsonl,19.137587,0.999998,0.767642,0.175879
world-log-374.jsonl,22.239223,0.999998,0.733432,0.184541
world-log-385.jsonl,21.388939,0.999999,0.877316,0.191055
world-log-831.jsonl,18.267273,0.997023,0.868059,0.193791
world-log-305.jsonl,19.778750,0.999998,0.768077,0.194385
world-log-256.jsonl,16.664202,0.999999,0.786262,0.201530
world-log-220.jsonl,19.587984,0.999998,0.875368,0.201867
world-log-685.jsonl,21.594353,0.999999,0.803666,0.204504
world-log-324.jsonl,22.266792,0.999999,0.766516,0.204965


# Display Model

In [ ]:
# !pip uninstall -q parlai
!pip install -q parlai
!pip install transformers

Create a new task named **french_blended_skill_talk** to be used in the next part

In [10]:
googledrive_data_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/"
data_path = "/content/dataset_french_bst/"
!rm -R $data_path
!mkdir $data_path
!cp -rv $googledrive_data_path* $data_path

task_path = "/usr/local/lib/python3.7/dist-packages/parlai/tasks/"

!rm -R $task_path'french_blended_skill_talk/'
!mkdir $task_path'french_blended_skill_talk'
!cp -ruv /content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_blended_skill_talk/* $task_path'french_blended_skill_talk/'


rm: cannot remove '/content/dataset_french_bst/': No such file or directory
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/test.txt' -> '/content/dataset_french_bst/test.txt'
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/train.txt' -> '/content/dataset_french_bst/train.txt'
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/valid.txt' -> '/content/dataset_french_bst/valid.txt'
rm: cannot remove '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_blended_skill_talk/': No such file or directory
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_blended_skill_talk/agents.py' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_blended_skill_talk/agents.py'
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_blended_skill_talk/build.py' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_blended_skill_talk/build.py'
'/content/driv

In [24]:
model_path = "/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m/"
from parlai.scripts.display_model import DisplayModel
from parlai.scripts.eval_model import EvalModel
import json

import time
start_time = time.time()

output_path = "/content/drive/MyDrive/colabs/blender-models/output-files/"
filename = "world-log-758.jsonl"
with open(output_path + filename) as f:
    params_line = f.readlines()[-1]
    params = json.loads(params_line)

print(params)
params.pop('world_logs', None)
params.pop('display_examples', None)
params['dict_language'] = 'french'

DisplayModel.main(**params)

{'beam_block_ngram': 3, 'beam_context_block_ngram': 3, 'beam_length_penalty': 0.8, 'beam_min_length': 20, 'beam_size': 20, 'datatype': 'valid', 'display_examples': True, 'inference': 'topk', 'model_file': '/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m/model', 'num_examples': 20, 'skip_generation': False, 'task': 'french_blended_skill_talk', 'temperature': 0.5, 'topk': 20, 'topp': 0.3, 'world_logs': '/content/drive/MyDrive/colabs/blender-models/output-files/world-log-758'}
10:49:33 | Overriding opt["beam_block_ngram"] to 3 (previously: -1)
10:49:33 | Overriding opt["beam_context_block_ngram"] to 3 (previously: -1)
10:49:33 | Overriding opt["beam_length_penalty"] to 0.8 (previously: 0.65)
10:49:33 | Overriding opt["beam_min_length"] to 20 (previously: 1)
10:49:33 | Overriding opt["beam_size"] to 20 (previously: 1)
10:49:33 | Overriding opt["datatype"] to valid (previously: train)
10:49:33 | Overriding opt["inference"] to topk (previously: greedy)
10:49:33 | Overri